# Prova

In [3]:
import numpy as np
import pandas as pd

In [9]:
! pip install pyarrow
! pip install caveclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 14.9 MB/s eta 0:00:0000:0100:01
  Using cached caveclient-5.13.0.tar.gz (63 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 16.0 MB/s eta 0:00:0000:0100:01
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached cachetools-5.3.2-py3-none-any.whl.metadata (5.2 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jsonschema-4.20.0-py3-none-any.whl.metadata (8.1 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 5.8 MB/s eta 0:00:00
  Using cached jsonschema_specifications-2023.11.1-py3-none-any.whl.metadata (2.8 kB)
  Us

In [2]:
from caveclient import CAVEclient

In [3]:
client = CAVEclient()

In [4]:
# client.auth.get_new_token()

New Tokens need to be acquired by hand. Please follow the following steps:
                1) Go to: https://global.daf-apis.com/auth/api/v1/create_token to create a new token.
                2) Log in with your Google credentials and copy the token shown afterward.
                3a) Save it to your computer with: client.auth.save_token(token="PASTE_YOUR_TOKEN_HERE")
                or
                3b) Set it for the current session only with client.auth.token = "PASTE_YOUR_TOKEN_HERE"
                Note: If you need to save or load multiple tokens, please read the documentation for details.
                Warning! Creating a new token by finishing step 2 will invalidate the previous token!


In [7]:
client.auth.save_token( "64ea0271652fd86dc7fb6376cd9d52b9")

In [8]:
client = CAVEclient('minnie65_public')

In [9]:
client.materialize.get_tables()

['synapses_pni_2',
 'baylor_gnn_cell_type_fine_model_v2',
 'nucleus_alternative_points',
 'connectivity_groups_v507',
 'proofreading_status_public_release',
 'allen_column_mtypes_v1',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'aibs_soma_nuc_exc_mtype_preds_v117',
 'baylor_log_reg_cell_type_coarse_v1',
 'apl_functional_coreg_forward_v5',
 'nucleus_detection_v0',
 'aibs_soma_nuc_metamodel_preds_v117',
 'coregistration_manual_v3']

In [10]:
matched_df = client.materialize.query_table('coregistration_manual_v3')
matched_df

,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position
0,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,864691135738685297,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]"
1,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,864691135614842827,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]"
2,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,864691135502985397,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]"
3,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,864691135387371905,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]"
4,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,864691135661410544,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13920,2323,2023-04-05 22:39:00.197594+00:00,t,293075,9,4,4693,4,3.21152,6.692787,293075,2020-09-28 22:41:16.438872+00:00,t,287.333417,91487065153289270,864691136144304948,"[194528, 118128, 23566]","[nan, nan, nan]","[nan, nan, nan]"
13921,10023,2023-04-05 22:39:40.658776+00:00,t,293075,9,4,7533,6,10.42720,-0.232141,293075,2020-09-28 22:41:16.438872+00:00,t,287.333417,91487065153289270,864691136144304948,"[194528, 118128, 23566]","[nan, nan, nan]","[nan, nan, nan]"
13922,2840,2023-04-05 22:39:00.636805+00:00,t,395416,9,3,2456,2,4.95407,3.168314,395416,2020-09-28 22:44:46.280352+00:00,t,282.205061,97825062761246370,864691135394864117,"[240528, 154032, 25770]","[nan, nan, nan]","[nan, nan, nan]"
13923,4554,2023-04-05 22:39:07.141941+00:00,t,395416,9,3,4759,4,6.51528,4.102491,395416,2020-09-28 22:44:46.280352+00:00,t,282.205061,97825062761246370,864691135394864117,"[240528, 154032, 25770]","[nan, nan, nan]","[nan, nan, nan]"


In [11]:
matched_df.sample(1)

,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position
2469,5780,2023-04-05 22:39:09.977588+00:00,t,258291,4,7,3568,4,3.18062,5.723602,258291,2020-09-28 22:44:45.927662+00:00,t,281.050644,88323082670416245,864691135992618689,"[171248, 137712, 21493]","[nan, nan, nan]","[nan, nan, nan]"


In [6]:
import datajoint as dj
from microns_phase3 import nda, utils

[2023-11-20 11:03:19,337][INFO]: Connecting root@localhost:3306


OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([Errno 61] Connection refused)")

# Codice Riley

In [4]:
nda_df = pd.read_csv('/Users/mattiamartino/Desktop/Uni/Thesis/code/Rileys_code/all_functional_neuron_synapses.csv')

In [6]:
nda_df.head()

,Input Depth,Output Depth
0,483.934154,444.215143
1,406.586430,444.215143
2,557.534745,444.215143
3,330.643998,452.832360
4,330.643998,452.832360
